In [29]:
import pickle
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from sklearn.metrics import classification_report
from collections import Counter

In [30]:
%pip install imbalanced-learn


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\km201\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [31]:
#internal metrics are generated now and everything is stored in dataframe should be ready for clustering
target_features = [
     'snake_case_var_ratio',
     'snake_case_class_ratio',
     'snake_case_method_ratio',
     'upper_camel_case_var_ratio',
     'upper_camel_case_class_ratio',
     'upper_camel_case_method_ratio',
     'lower_camel_case_var_ratio',
     'lower_camel_case_class_ratio',
     'lower_camel_case_method_ratio',
     'func_decorators_avg',
     'class_decorators_avg',
     'class_parents_avg',
     'comprehensions_avg',
     'generators_avg',
     'lambda_avg',
     'comment_density',
     'ds_density',
]

In [32]:


#full dataset
combined_df = pd.read_csv("data/bigquery_metric_dedup_file0_20220530.csv")
os.listdir("data")
bq_metric_files = [ 
    #'bigquery_metric_dedup_file0_20220530.csv',
    'py150k_metric_20220527.csv',
    'bigquery_metric_dedup_file1_20220530.csv',
    'bigquery_metric_dedup_file2_20220530.csv',
    'bigquery_metric_dedup_file3_20220530.csv',
    'bigquery_metric_dedup_file4_20220530.csv',
    'bigquery_metric_dedup_file5_20220530.csv',
    'bigquery_metric_dedup_file6_20220530.csv',
    'bigquery_metric_dedup_file7_20220530.csv',
    'bigquery_metric_dedup_file8_20220530.csv',
    'bigquery_metric_dedup_file9_20220530.csv' 
 ]

for file in bq_metric_files:
    temp_df = pd.read_csv(f'data/{file}')
    combined_df = pd.concat((combined_df[target_features], temp_df[target_features]))
#os.listdir("data")
display(combined_df)

,snake_case_var_ratio,snake_case_class_ratio,snake_case_method_ratio,upper_camel_case_var_ratio,upper_camel_case_class_ratio,upper_camel_case_method_ratio,lower_camel_case_var_ratio,lower_camel_case_class_ratio,lower_camel_case_method_ratio,func_decorators_avg,class_decorators_avg,class_parents_avg,comprehensions_avg,generators_avg,lambda_avg,comment_density,ds_density
0,0.172414,0.0,0.000000,0.000000,0.000000,0.0,0.034483,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.209677,0.0,1.000000,0.056452,1.000000,0.0,0.000000,0.0,0.000000,0.250000,0.0,1.000000,0.005882,0.000000,0.017647,0.047059,0.058824
2,0.169231,0.0,0.800000,0.038462,1.000000,0.0,0.000000,0.0,0.200000,0.000000,0.0,1.000000,0.000000,0.000000,0.022222,0.251852,0.000000
3,0.545455,0.0,0.000000,0.018182,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.186441,0.067797
4,0.000000,0.0,0.400000,0.000000,0.000000,0.0,0.282609,0.0,0.600000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.157895,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87927,0.137255,0.0,0.321429,0.009804,0.333333,0.0,0.000000,0.0,0.000000,0.071429,0.0,1.000000,0.003584,0.000000,0.000000,0.050179,0.010753
87928,0.378378,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.045455,0.000000
87929,0.029674,0.0,0.414634,0.029674,0.666667,0.0,0.023739,0.0,0.439024,0.000000,0.0,0.333333,0.000000,0.000000,0.001992,0.039841,0.005976
87930,0.205714,0.0,0.833333,0.030476,1.000000,0.0,0.000000,0.0,0.000000,0.375000,0.0,1.000000,0.002304,0.006912,0.004608,0.002304,0.000000


In [33]:
#only py150 and first manifest
#py150k_df = pd.read_csv("data/py150k_metric_20220527.csv")
#bq_df = pd.read_csv("data/bigquery_metric_20220526.csv")
#combined_df = pd.concat([py150k_df[target_features], bq_df[target_features]])

In [34]:

with open("data/BigQuery/clusters/feature_set_10/full_feature_clusterer_preds.pickle", "rb") as file:
    cluster_pred = pickle.load(file)
    labels = cluster_pred.labels_
    cluster_num = len(np.unique(labels))

    #X = combined_df[target_features].to_numpy()

In [35]:
from sklearn.linear_model import SGDClassifier
# from sklearn.preprocessing import StandardScaler
# from sklearn.pipeline import make_pipeline
# X = np.array([[-1, -1], [-2, -1], [1, 1], [2, 1]])
# Y = np.array([1, 1, 2, 2])
# # Always scale the input. The most convenient way is to use a pipeline.
# clf = make_pipeline(StandardScaler(),
#                     SGDClassifier(max_iter=1000, tol=1e-3))
clf = SGDClassifier(max_iter=1000, tol=1e-3)


In [36]:
non_outliers_bool = labels != -1

In [37]:
non_outliers_bool[108479]

False

In [38]:
train_df = combined_df[non_outliers_bool].sample(frac=0.8,random_state=1234)
test_df = combined_df[non_outliers_bool].drop(train_df.index)

In [39]:
train_df.index

Int64Index([63750, 57784, 25925, 77572, 87451,  1242, 87060, 41444, 66871,
            53240,
            ...
            30195, 67326, 84614, 57365, 79604, 25599, 62086, 67198, 25821,
            64955],
           dtype='int64', length=218394)

In [40]:
train_X = train_df[target_features].to_numpy()
test_X = test_df[target_features].to_numpy()


In [41]:
#issue here with the indexes since they are not reset after concatting
train_y = [labels[idx] for idx in train_df.index]
test_y = [labels[idx] for idx in test_df.index]
display(combined_df)

,snake_case_var_ratio,snake_case_class_ratio,snake_case_method_ratio,upper_camel_case_var_ratio,upper_camel_case_class_ratio,upper_camel_case_method_ratio,lower_camel_case_var_ratio,lower_camel_case_class_ratio,lower_camel_case_method_ratio,func_decorators_avg,class_decorators_avg,class_parents_avg,comprehensions_avg,generators_avg,lambda_avg,comment_density,ds_density
0,0.172414,0.0,0.000000,0.000000,0.000000,0.0,0.034483,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.209677,0.0,1.000000,0.056452,1.000000,0.0,0.000000,0.0,0.000000,0.250000,0.0,1.000000,0.005882,0.000000,0.017647,0.047059,0.058824
2,0.169231,0.0,0.800000,0.038462,1.000000,0.0,0.000000,0.0,0.200000,0.000000,0.0,1.000000,0.000000,0.000000,0.022222,0.251852,0.000000
3,0.545455,0.0,0.000000,0.018182,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.186441,0.067797
4,0.000000,0.0,0.400000,0.000000,0.000000,0.0,0.282609,0.0,0.600000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.157895,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87927,0.137255,0.0,0.321429,0.009804,0.333333,0.0,0.000000,0.0,0.000000,0.071429,0.0,1.000000,0.003584,0.000000,0.000000,0.050179,0.010753
87928,0.378378,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.045455,0.000000
87929,0.029674,0.0,0.414634,0.029674,0.666667,0.0,0.023739,0.0,0.439024,0.000000,0.0,0.333333,0.000000,0.000000,0.001992,0.039841,0.005976
87930,0.205714,0.0,0.833333,0.030476,1.000000,0.0,0.000000,0.0,0.000000,0.375000,0.0,1.000000,0.002304,0.006912,0.004608,0.002304,0.000000


In [42]:
# corrected label setup
combined_df['labels'] = labels
filtered_df = combined_df[combined_df['labels'] != -1]
#filtered_df = combined_df

train_df = filtered_df.sample(frac=0.8,random_state=1234)
test_df = filtered_df.drop(train_df.index)

train_X = train_df[target_features].to_numpy()
test_X = test_df[target_features].to_numpy()

train_y = train_df['labels'].to_list()
test_y = test_df['labels'].to_list()
train_df.index


Int64Index([63750, 57784, 25925, 77572, 87451,  1242, 87060, 41444, 66871,
            53240,
            ...
            30195, 67326, 84614, 57365, 79604, 25599, 62086, 67198, 25821,
            64955],
           dtype='int64', length=218394)

In [43]:
result = {}

In [44]:

clf = SGDClassifier("hinge",max_iter=1000000, tol=1e-3, verbose=0, early_stopping=True, class_weight='balanced').fit(train_X, train_y)

preds = clf.predict(train_X)
report = classification_report(train_y, preds, output_dict=True)
result["svm_train"] = report.copy()

preds = clf.predict(test_X)
report = classification_report(test_y, preds, output_dict=True)
result["svm_test"] = report.copy()

C:\Users\km201\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\km201\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\km201\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1327

In [45]:
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(random_state=0)
X_resampled, y_resampled = rus.fit_resample(train_X, train_y)
print(sorted(Counter(y_resampled).items()))


[(0, 423), (1, 423), (2, 423), (3, 423), (4, 423), (5, 423), (6, 423), (7, 423), (8, 423), (9, 423), (10, 423), (11, 423), (12, 423), (13, 423), (14, 423), (15, 423), (16, 423), (17, 423), (18, 423), (19, 423), (20, 423), (21, 423), (22, 423), (23, 423), (24, 423), (25, 423), (26, 423), (27, 423), (28, 423), (29, 423), (30, 423), (31, 423), (32, 423), (33, 423), (34, 423), (35, 423), (36, 423), (37, 423), (38, 423), (39, 423), (40, 423), (41, 423), (42, 423), (43, 423), (44, 423), (45, 423), (46, 423), (47, 423), (48, 423), (49, 423), (50, 423), (51, 423), (52, 423), (53, 423), (54, 423), (55, 423), (56, 423), (57, 423), (58, 423), (59, 423), (60, 423), (61, 423), (62, 423), (63, 423), (64, 423), (65, 423), (66, 423), (67, 423), (68, 423), (69, 423)]


In [46]:
# SVM with Undersampling
clf = SGDClassifier("hinge",max_iter=1000000, tol=1e-3, verbose=0, early_stopping=True).fit(X_resampled, y_resampled)

preds = clf.predict(train_X)
report = classification_report(train_y, preds, output_dict=True)
result["svm_undersample_train"] = report.copy()

preds = clf.predict(test_X)
report = classification_report(test_y, preds, output_dict=True)
result["svm_undersample_test"] = report.copy()

C:\Users\km201\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\km201\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\km201\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1327

In [47]:
from sklearn.linear_model import LogisticRegression
#clf = SGDClassifier("log_loss",max_iter=1000000, tol=0.0001, verbose=0, early_stopping=True, class_weight = 'balanced').fit(train_X, train_y)
clf = LogisticRegression(max_iter=100000, tol = 0.0001, class_weight = 'balanced', C = 1).fit(train_X, train_y)

preds = clf.predict(train_X)
report = classification_report(train_y, preds, output_dict=True)
result["lr_train"] = report.copy()

preds = clf.predict(test_X)
report = classification_report(test_y, preds, output_dict=True)
result["lr_test"] = report.copy()

'clf = LogisticRegression(max_iter=100000, tol = 0.0001, class_weight = \'balanced\', C = 1).fit(train_X, train_y)\n\npreds = clf.predict(train_X)\nreport = classification_report(train_y, preds, output_dict=True)\nresult["lr_train"] = report.copy()\n\npreds = clf.predict(test_X)\nreport = classification_report(test_y, preds, output_dict=True)\nresult["lr_test"] = report.copy()'

In [48]:
from sklearn import tree
clf = tree.DecisionTreeClassifier(max_depth=17)
clf = clf.fit(train_X, train_y)

preds = clf.predict(train_X)
report = classification_report(train_y, preds, output_dict=True)
result["tree_train"] = report.copy()

preds = clf.predict(test_X)
report = classification_report(test_y, preds, output_dict=True)
result["tree_test"] = report.copy()

In [49]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()
clf = clf.fit(train_X, train_y)

preds = clf.predict(train_X)
report = classification_report(train_y, preds, output_dict=True)
result["r_forest_train"] = report.copy()

preds = clf.predict(test_X)
report = classification_report(test_y, preds, output_dict=True)
result["r_forest_test"] = report.copy()

In [50]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(train_X, train_y)

preds = clf.predict(train_X)
report = classification_report(train_y, preds, output_dict=True)
result["nb_train"] = report.copy()

preds = clf.predict(test_X)
report = classification_report(test_y, preds, output_dict=True)
result["nb_test"] = report.copy()


C:\Users\km201\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\km201\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\km201\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1327

In [51]:
for key in result.keys():
    print(f"{key}:{result[key]['macro avg']['f1-score']}")

svm_train:0.6579930424998811
svm_test:0.6569836623332376
svm_undersample_train:0.6662465774161523
svm_undersample_test:0.6668263236165572
tree_train:0.999870457640896
tree_test:0.9995712234401044
r_forest_train:0.9999979581301046
r_forest_test:1.0
nb_train:0.30330135146703086
nb_test:0.2995641792490101
